# Setting

In [3]:
import numpy as np
import random 
import os
import sys
# import scipy.stats
import pandas as pd
from tqdm import tqdm

import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [4]:
import tensorflow as tf 

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate,AveragePooling1D
from keras.models import Model, load_model

from keras.optimizers import Adam, RMSprop

In [5]:
print("Tensorflow GPU availability: ", tf.config.list_physical_devices('GPU'))

Tensorflow GPU availability:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
def set_seed(seed=42):    
    '''랜덤시드 고정. Hyperparm tuning 제외 모든 학습환경에서 
    같은 성능이 나오게 합니다.'''
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

# Import data, model

In [19]:
MC_DROPOUT_MODE = 'ON' #ON/OFF
targets_abs_max  = 0.0030516885275461087
nb_dropout_wc = 1000
nb_dropout = 5

In [8]:
model_path = './output/'
data_path = './data/'
pred_path = './output/'

In [9]:
# data, metadata import
test_adc_info = pd.read_csv(data_path + 'test_adc_info.csv', index_col='planet_id')
train_labels = pd.read_csv(data_path + 'train_labels.csv', index_col='planet_id')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='planet_id')
wavelengths = pd.read_csv(data_path + 'wavelengths.csv')

In [10]:
# testset_1d = np.load(pred_path + 'testset_1d.npy')
testset_2d = np.load(pred_path + 'testset_2d.npy')
testset_2d.shape

(1, 40, 283, 1)

# Make prediction

## predict wl

1D CNN (Predict mean)

In [11]:
# cnn1d_model = load_model(model_path+'/model_cnn1d.h5', compile=False) # 현 tensorflow 버전 때문에 compile=False. Kaggle에선 떼고 사용할 것.

In [12]:
# cnn1d_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [13]:
# predictions_1d = cnn1d_model.predict(testset_1d)
# predictions_1d

### Postprocessing

### 2D CNN (Predict shape)

In [14]:
cnn2d_model = load_model(model_path+'/model_cnn2d.h5', compile=False)

In [15]:
cnn2d_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [16]:
predictions_2d = cnn2d_model.predict(testset_2d)
predictions_2d.shape, predictions_2d.max(), predictions_2d.min()

1/1 [==============================] - 3s 3s/step


((1, 283), 0.0139748035, -0.010044133)

Denormalize prediction


In [ ]:
predictions_2d = predictions_2d * targets_abs_max
predictions_2d.max(), predictions_2d.min()

(4.2646745e-05, -3.0651565e-05)

### Sum 1D result and 2D result

In [ ]:
wl_prediction = predictions_2d + predictions_1d

## predict Uncertainty (Sigma)

### 1D CNN uncertainty 계산

In [ ]:
def unstandardizing (data, min_train_valid, max_train_valid) : 
    return data * (max_train_valid - min_train_valid) + min_train_valid

In [ ]:
def MC_dropout_wc(model, data, nb_dropout):
    predictions = np.zeros((nb.dropout, data.shape[0]))
    for i in range(nb_dropout):
        predictions[i,:] = model.prediction(data, verbose=0).flatten()
    return predictions

In [ ]:
if MC_DROPOUT_MODE=='ON':
    print('Running...')
    prediction_valid_wc = MC_dropout_wc(cnn1d_model, testset, nb_dropout_wc)
    unstandardizing(prediction_valid_wc, min_train_valid_wc, max_train_valid_wc)
    spectre_valid_wc, spectre_valid_std_wc = spectre_valid_wc_all.mean(axis = 0), spectre_valid_wc_all.std(axis = 0)
    print('Done.')
else:
    spectre_valid_wc = cnn1d_model.predict(testset).flatten()
    spectre_valid_wc = unstandardizing(spectre_valid_wc, min_train_valid_wc, max_train_valid_wc)
    spectre_valid_std_wc = 0.1*np.abs(spectre_valid_wc)

spectre_valid_wc, spectre_valid_std_wc

### 2D CNN uncertainty 계산

In [ ]:
if MC_DROPOUT_MODE=='ON':
    print('Running...')
    prediction_valid_wc = MC_dropout_WC(model_wc, valid_wc, nb_dropout_wc)
    spectre_valid_wc_all = unstandardizing(prediction_valid_wc, min_train_valid_wc, max_train_valid_wc)
    spectre_valid_wc, spectre_valid_std_wc = spectre_valid_wc_all.mean(axis=0), spectre_valid_wc_all.std(axis=0)
    print('Done')
else:
    spectre_valid_wc = model_wc.predict(valid_wc).flatten()
    spectre_valid_wc = unstandardizing(spectre_valid_wc, min_train_valid_wc, max_train_valid_wc)
    spectre_valid_std_wc = 0.1*np.abs(spectre_valid_wc)
    
spectre_valid_wc, spectre_valid_std_wc

# Validation Score (GLL)

In [ ]:
def calculate_score(df, ground_truth, naive_mean, naive_sigma, sigma_true):
    planet_id = df['planet_id']
    y_pred = df.iloc[:, 1:284].values   # wl만 뽑기
    sigma_pred = np.clip(df.iloc[:, 284:].values, a_min=10**-15, a_max=None) # sigma만 뽑기

    y_true = ground_truth.drop_fts['planet_id'] # planet_id 삭제

    GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
    GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
    GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

    submit_score = (GLL_pred - GLL_mean) / (GLL_true - GLL_mean)
    return float(np.clip(submit_score, 0.0, 1.0))

In [ ]:
naive_mean = 0.5
naive_sigma = 0.1
sigma_true = 1e-6

score = calculate_score(y_pred, y_truth, naive_mean, naive_sigma, sigma_true)
print(f'Validation Score: {score}')

# Submission

In [ ]:
def postprocessing(pred, index, sigma_pred):
    return pd.concat([pd.DataFrame(pred.clip(0, None), index=index, columns=wavelengths.columns),
    pd.DataFrame(sigma_pred, index=index, columns=[f'sigma_{i}' for i in range(1, 284)])])

In [ ]:
submission = postprocessing(predictions_1d, test_adc_info.index, predictions_2d)

submission.to_csv('submission.csv')